# Basics of Agents
We shall look at the basics of the Chat Completions API and prompting which are the underpinnings of 
Agentic operations such as tool calls.

In [ ]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
from openai import OpenAI
client = OpenAI()

# https://platform.openai.com/docs/api-reference/chat/create

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Your name is Cathy and you are a part of a duo of comedians."},
        {
            "role": "user",
            "name": "Joe",
            "content": "Tell me a joke !"
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Sure, here's one for you:\n\nWhy don’t skeletons fight each other? \n\nThey don’t have the guts!", refusal=None, role='assistant', function_call=None, tool_calls=None)


In [6]:
print(completion.choices[0].message.content)

Sure, here's one for you:

Why don’t skeletons fight each other? 

They don’t have the guts!


In [ ]:
import agentops

from autogen import ConversableAgent

In [ ]:
gentops.init(api_key="16947674-224c-4d71-a7cd-6c29326079be")
langfuse = Langfuse()
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)